In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
dataset_df = pd.read_csv('data.csv',encoding="utf-8",converters={"tags":literal_eval})
dataset_df

,Unnamed: 0,name,steam_appid,release_date,is_free,required_age,rate,price,owners,tags
0,0,Dota 2,570,2013,True,0,0.829342,0,350000000,"[Free to Play, MOBA, Multiplayer, Strategy, e-..."
1,1,Counter-Strike: Global Offensive,730,2012,True,0,0.882848,0,75000000,"[FPS, Shooter, Multiplayer, Competitive, Actio..."
2,2,New World,1063730,2021,False,0,0.680567,3999,75000000,"[Massively Multiplayer, Open World, MMORPG, RP..."
3,3,Team Fortress 2,440,2007,True,0,0.936167,0,75000000,"[Free to Play, Hero Shooter, Multiplayer, FPS,..."
4,4,Apex Legends™,1172470,2020,True,0,0.858303,0,75000000,"[Free to Play, Multiplayer, Battle Royale, Sho..."
...,...,...,...,...,...,...,...,...,...,...
48858,48858,VEHICLES FURY,749290,2017,False,0,0.166667,1499,10000,"[Action, Indie, Racing, Sports]"
48859,48859,Crawler,1506770,2021,False,0,0.416667,399,10000,"[Adventure, Action RPG, Exploration, Dungeon C..."
48860,48860,Coins Collector Simulator,1868280,2022,False,0,0.625000,499,10000,"[Casual, Racing, Simulation, Sports, Strategy,..."
48861,48861,You Arrive in a Town,1466190,2021,False,0,0.473684,499,10000,"[Indie, Action, Casual, RPG, Action RPG, Arcad..."


In [3]:
'''
dictionary
key: year
val: dictionary mapping tag to number
'''
year_tag_dict = {}
for index, row in dataset_df.iterrows():
    year = row["release_date"]
    tags = row["tags"]
    if year in year_tag_dict:
        tag_count_dict = year_tag_dict[year]
        for tag in tags:
            if tag in tag_count_dict:
                tag_count_dict[tag] += 1
            else:
                tag_count_dict[tag] = 1
    else:
        year_tag_dict[year] = {}
        for tag in tags:
            year_tag_dict[year][tag] = 1

In [4]:
rank = pd.read_csv("players.csv")
rank

,Unnamed: 0,Name,players
0,0,Counter-Strike: Global Offensive,"616,202"
1,1,Dota 2,"450,685"
2,2,Apex Legends™,"209,117"
3,3,PUBG: BATTLEGROUNDS,"200,559"
4,4,Lost Ark,"153,282"
...,...,...,...
245,245,Green Hell,"2,030"
246,246,Incremental Epic Hero 2,"2,018"
247,247,Cuphead,"1,995"
248,248,Last Epoch,"1,971"


In [21]:
dataset_df['name']

0                                          Dota 2
1                Counter-Strike: Global Offensive
2                             PUBG: BATTLEGROUNDS
3                                       New World
4                                 Team Fortress 2
                           ...                   
56341                                     Crawler
56342                   Coins Collector Simulator
56343                        You Arrive in a Town
56344    Hidden Treasures in the Forest of Dreams
56345                              The Chaos Gene
Name: name, Length: 56346, dtype: object

In [34]:
names = rank["Name"].tolist()
print(names[0] == dataset_df.name[1])
names[0] in dataset_df.name.values

True


True

In [35]:

for name in names:
    if name not in dataset_df.name.values:
        print(name)
    else:
        count += 1
        
print(count)

MIR4
EA SPORTS™ FIFA 23
Yu-Gi-Oh! Master Duel
GUNDAM EVOLUTION
Shatterline
Brotato
Omega Strikers
Construction Simulator
Terra Invicta
Slime Rancher 2
Pathfinder: Wrath of the Righteous - Enhanced Edition
Idle Skilling
eFootball™ 2023
Dome Keeper
BONELAB
ヘブンバーンズレッド
Farmer Against Potatoes Idle
Lost Light
Incremental Epic Hero 2
231
